# SQL Parser using Lark
### Lark
- Ref: [Lark Documentation](https://lark-parser.readthedocs.io/en/latest/)
  - Repo: [lark-parser](https://github.com/lark-parser/lark)

### Lark SQL Parser
- Ref: [sql_to_ibis](https://github.com/zbrookle/sql_to_ibis)
- Ref: [ibis](https://github.com/ibis-project/ibis)
  - Docs: https://ibis-project.org/docs/dev/

`sql_to_ibis` is a [Python](https://www.python.org/) package that translates SQL syntax into [ibis](https://github.com/ibis-project/ibis) expressions. This provides the capability of using only one SQL dialect to target many different backends.

- Ref: From `sql_to_ibis` project `sql_select_query.py`
- Ref: SQL grammar file <https://github.com/zbrookle/sql_to_ibis/tree/main/sql_to_ibis/grammar>

## Transform into Firebase Update
### Transformers
* **Transformers** -  work bottom-up (or depth-first), starting with visiting the leaves and working their way up until ending at the root of the tree.
  * For each node visited, the transformer will call the appropriate method (callbacks), according to the node’s `data`, and use the returned value to replace the node, thereby creating a new tree structure.
  * Transformers can be used to implement map & reduce patterns. Because nodes are reduced from leaf to root, at any point the callbacks may assume the children have already been transformed

In [1]:
from firesql.firebase import FirebaseClient
from firesql.sql.sql_fire_client import FireSQLClient

firebaseClient = FirebaseClient()
firebaseClient.connect(credentials_json='../credentials/credentials.json')

# create FireSQLClient that is using the FireSQLAbstractClient interface.
client = FireSQLClient(firebaseClient)

In [2]:
import os
from lark import Lark, tree

_ROOT = "../firesql/sql"
GRAMMAR_PATH = os.path.join(_ROOT, "grammar", "firesql.lark")
with open(file=GRAMMAR_PATH) as sql_grammar_file:
    _GRAMMAR_TEXT = sql_grammar_file.read()
parser = Lark(_GRAMMAR_TEXT, parser="lalr")

In [3]:
from firesql.sql.sql_transformer import SelectTransformer

company = 'bennycorp'

sql1 = """
  UPDATE Companies/{}/Users
  SET state = 'INACTIVE'
  WHERE state = 'ACTIVE' AND docid = '0r6YWowe9rW65yB1qTKsCe83cCm2'
""".format(company)

ast = parser.parse(sql1)
sqlCommand = SelectTransformer().transform(ast)
sqlCommand

SQL_Update(table=SQL_SelectFrom(part='Companies/bennycorp/Users', alias=None), sets=[SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='state', func=None), right=SQL_ValueString(value='INACTIVE'))], where=SQL_BinaryExpression(operator='and', left=SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='state', func=None), right=SQL_ValueString(value='ACTIVE')), right=SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='docid', func=None), right=SQL_ValueString(value='0r6YWowe9rW65yB1qTKsCe83cCm2'))))

In [4]:
from firesql.sql import SQLFireUpdate  

fireQuery = SQLFireUpdate()
queries = fireQuery.update_generate(sqlCommand, options={})

In [5]:
print(fireQuery.collections)
print(fireQuery.sets)
print(queries)

{'Companies/bennycorp/Users': 'Companies/bennycorp/Users'}
{'Companies/bennycorp/Users': {'state': 'INACTIVE'}}
{'Companies/bennycorp/Users': [['state', '==', 'ACTIVE'], ['docid', '==', '0r6YWowe9rW65yB1qTKsCe83cCm2']]}


In [6]:
fireQueries = fireQuery.firebase_queries(queries)
fireQueries

{'Companies/bennycorp/Users': [['state', '==', 'ACTIVE'],
  ['docid', '==', '0r6YWowe9rW65yB1qTKsCe83cCm2']]}

In [7]:
filterQueries = fireQuery.filter_queries(queries)
filterQueries

{'Companies/bennycorp/Users': []}

In [8]:
documents = fireQuery.execute(client, fireQueries)

In [9]:
filterDocs = fireQuery.filter_documents(documents, filterQueries)
filterDocs

{'Companies/bennycorp/Users': {'0r6YWowe9rW65yB1qTKsCe83cCm2': {'access': {'hasAccess': True},
   'preferredSeats': ['10eB0ir2u522CmwXgFvz'],
   'groups': ['NLu0L6sEOUJHFBRCiylr'],
   'firstName': 'Benny',
   'vaccination': None,
   'emailVerified': True,
   'events': [{'date': DatetimeWithNanoseconds(2022, 2, 25, 14, 46, 29, 68000, tzinfo=datetime.timezone.utc),
     'event': 'CREATION'},
    {'new_state': 'ACTIVE',
     'event': 'STATE_CHANGE',
     'old_state': 'INACTIVE',
     'date': DatetimeWithNanoseconds(2022, 2, 25, 14, 47, 18, 78000, tzinfo=datetime.timezone.utc)},
    {'event': 'ACCESS_POLICY_CHANGE',
     'date': DatetimeWithNanoseconds(2022, 2, 28, 19, 7, 40, 865000, tzinfo=datetime.timezone.utc)}],
   'state': 'ACTIVE',
   'roles': [],
   'lastName': 'Real',
   'externalId': None,
   'assignedSeats': [],
   'email': 'btscheung+real@gmail.com'}}}

In [10]:
selectDocs = fireQuery.update_post_process(filterDocs)

In [11]:
fireQuery.select_fields()

['access',
 'assignedSeats',
 'docid',
 'email',
 'emailVerified',
 'events',
 'externalId',
 'firstName',
 'groups',
 'lastName',
 'preferredSeats',
 'roles',
 'state',
 'vaccination']

In [12]:
from firesql.sql import DocPrinter

docPrinter = DocPrinter()
docPrinter.printCSV(selectDocs, fireQuery.select_fields())

"access","assignedSeats","docid","email","emailVerified","events","externalId","firstName","groups","lastName","preferredSeats","roles","state","vaccination"
"{\"hasAccess\": true}","","0r6YWowe9rW65yB1qTKsCe83cCm2","btscheung+real@gmail.com",True,"{\"date\": \"2022-02-25T14:46:29\", \"event\": \"CREATION\"},{\"new_state\": \"ACTIVE\", \"event\": \"STATE_CHANGE\", \"old_state\": \"INACTIVE\", \"date\": \"2022-02-25T14:47:18\"},{\"event\": \"ACCESS_POLICY_CHANGE\", \"date\": \"2022-02-28T19:07:40\"}",None,"Benny","NLu0L6sEOUJHFBRCiylr","Real","10eB0ir2u522CmwXgFvz","","INACTIVE",None


In [13]:
selectDocs = fireQuery.update_execute(client, filterDocs)